In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import numpy as np


In [13]:
data = pd.read_csv('penguins_size.csv')
data.head() 

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [14]:
#drop de la colonne sex (créant redondance avec les tailles et taille des ailles)
data.drop(columns=['sex'], inplace=True)

In [15]:
# Supprimer les enregistrements incomplets
data.dropna(inplace=True)

In [16]:
# Séparer les caractéristiques (X) de la cible (y)
X = data[['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm' , 'island', 'species']]
y = data['body_mass_g']

In [17]:
# Créer une ColumnTransformer pour gérer les transformations des différentes colonnes
# On définit les transformations pour les colonnes 'island' et 'species' comme OneHotEncoder
# On définit une transformation par défaut pour les autres colonnes comme StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['island', 'species']),
        ('num', StandardScaler(), ['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm'])
    ],
    remainder='passthrough'  # Permet de conserver les colonnes non transformées (ici 'body_mass_g')
)

# Créer un pipeline avec le préprocesseur et le modèle SVR
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svr', SVR())
])


In [18]:
# Définir les paramètres à rechercher pour le modèle SVR
param_grid = {
    'svr__kernel': ['linear', 'rbf', 'poly'],
    'svr__C': [0.1, 1, 10, 100],
    'svr__gamma': ['scale', 'auto']
}

In [19]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Créer une instance de GridSearchCV avec le pipeline et les paramètres à rechercher
grid_search = GridSearchCV(pipeline, param_grid, cv=KFold(n_splits=10, shuffle=True, random_state=42))

In [28]:
# Exécuter la recherche sur grille sur les données d'entraînement
y_pred = grid_search.predict(X_test)

In [29]:
# Afficher les meilleurs paramètres et le meilleur score
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score :", grid_search.best_score_)

# Évaluer les performances du meilleur modèle sur l'ensemble de test
test_score = grid_search.score(X_test, y_test)
print("Score sur l'ensemble de test avec le meilleur modèle :", test_score)

# Calculer le MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE) :", mse)

# Calculer le RMSE en prenant la racine carrée du MSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE) :", rmse)


Meilleurs paramètres : {'svr__C': 100, 'svr__gamma': 'scale', 'svr__kernel': 'linear'}
Meilleur score : 0.8068770339113727
Score sur l'ensemble de test avec le meilleur modèle : 0.8590113025366858
Mean Squared Error (MSE) : 93745.93464443964
Root Mean Squared Error (RMSE) : 306.17957907809534
